In [3]:
# FULL NEPAL LANDSLIDE SUSCEPTIBILITY

# Install and Import Necessary Modules
!pip install earthengine-api joblib requests shapely --quiet

import ee
import pandas as pd
import numpy as np
import requests
import joblib
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from shapely.geometry import Point
from google.colab import drive
from datetime import datetime, timedelta, UTC

# Configuration & Initialization

ee.Authenticate()
ee.Initialize(project="landslide-data-481212")
drive.mount("/content/drive")

OPENWEATHER_API_KEY = "YOUR API KEY" #API
GRID_SPACING_KM = 6 # Grid

# Rate Limiting Settings
MAX_WORKERS = 100

MODEL_PATH = "/content/drive/MyDrive/final_project/model_landslide/landslide_xgboost.pkl"
OUTPUT_PATH = "/content/drive/MyDrive/final_project/landslide_prediction/nepal_risk_full.csv"

# Loading Model
model = joblib.load(MODEL_PATH)

# Boundary of Nepal
nepal_fc = (
  ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
  .filter(ee.Filter.eq("country_na", "Nepal"))
)

nepal_geom = nepal_fc.geometry()
bounds = nepal_geom.bounds().coordinates().getInfo()[0]

min_lon_full, min_lat_full = bounds[0]
max_lon_full, max_lat_full = bounds[2]

# Grid Generation
step_deg = GRID_SPACING_KM / 111.0

lats = np.arange(min_lat_full, max_lat_full, step_deg)
lons = np.arange(min_lon_full, max_lon_full, step_deg)

grid_points = []
for lat in lats:
  for lon in lons:
    p = ee.Geometry.Point([lon, lat])
    if nepal_geom.contains(p).getInfo():
      grid_points.append((lat, lon))

df = pd.DataFrame(grid_points, columns=["lat", "lon"])
print(f" Total grid points for FULL run: {len(df)}")


# Static Data Extraction
dem = ee.Image("USGS/SRTMGL1_003")
terrain = ee.Terrain.products(dem)
landcover = ee.Image("MODIS/061/MCD12Q1/2022_01_01").select("LC_Type1")
end_date = ee.Date(datetime.now(UTC))
start_date = end_date.advance(-30, "day")
ndvi = (
  ee.ImageCollection("MODIS/061/MOD13A2")
  .filterDate(start_date, end_date)
  .select("NDVI")
  .mean()
)

static_image = ee.Image.cat([
  dem.rename("elevation"),
  terrain.select("slope"),
  terrain.select("aspect"),
  landcover.rename("landcover_class"),
  ndvi.rename("ndvi")
])

points_fc = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([lon, lat]))
  for lat, lon in grid_points
])

print(" Starting GEE batch extraction...")
data_extracted = static_image.sampleRegions(
  collection=points_fc,
  scale=250,
  tileScale=16
).getInfo()

static_records = [feature['properties'] for feature in data_extracted['features']]
static_df = pd.DataFrame(static_records)

static_df["ndvi"] = (static_df["ndvi"] / 10000).clip(-1, 1)

df_static = pd.concat([df, static_df], axis=1)
print(" GEE extraction complete.")


# Rainfall Data Extraction

def fetch_rain(lat, lon, api_key):
  url = (
    f"https://api.openweathermap.org/data/2.5/forecast"
    f"?lat={lat}&lon={lon}&appid={api_key}&units=metric"
  )
  try:
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    data = r.json()["list"]

    rain_vals = [d.get("rain", {}).get("3h", 0) for d in data[:56]]

    mean_7d = np.mean(rain_vals) if rain_vals else 0
    last_24h = sum(rain_vals[:8])

    return lat, lon, mean_7d, last_24h
  except:
    return lat, lon, np.nan, np.nan

print(" Starting OpenWeather rain data extraction...")

rain_results = []
input_data = [(lat, lon, OPENWEATHER_API_KEY) for lat, lon in zip(df_static.lat, df_static.lon)]

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
  futures = [executor.submit(fetch_rain, lat, lon, OPENWEATHER_API_KEY) for lat, lon in zip(df_static.lat, df_static.lon)]

  start_time = time.time()
  calls_made = 0

  for future in as_completed(futures):
    result = future.result()
    rain_results.append(result)
    calls_made += 1

    if calls_made % 500 == 0:
      print(f" {calls_made}/{len(df_static)} points processed in {time.time() - start_time:.2f} seconds.")

results_df = pd.DataFrame(rain_results, columns=['lat', 'lon', '7D_mean_rain', '24H_rain'])

df_static = df_static.merge(results_df[['lat', 'lon', '7D_mean_rain', '24H_rain']], on=['lat', 'lon'], how='left')


df_static["7D_mean_rain"] = df_static["7D_mean_rain"].clip(0, 300)
df_static["24H_rain"] = df_static["24H_rain"].clip(0, 150)
print(" Parallel rain data extraction complete.")


# Model Prediction & Output

features = [
  "elevation", "slope", "aspect", "landcover_class",
  "7D_mean_rain", "24H_rain", "ndvi"
]

X = df_static[features].astype(float)
X = X.fillna(X.median())

df_static["landslide_probability"] = model.predict_proba(X)[:, 1]

# Risk Level
def risk_level(p):
  if p >= 0.9:
    return "High"
  elif p >= 0.5:
    return "Moderate"
  else:
    return "Low"

df_static["risk_level"] = df_static["landslide_probability"].apply(risk_level)

# Output
df_static.to_csv(OUTPUT_PATH, index=False)

print("\n Full Analysis Complete")
print(" Grid points analyzed:", len(df_static))
print(" Saved full results at:", OUTPUT_PATH)
print("\nRisk Distribution:")
print(df_static['risk_level'].value_counts(normalize=True))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 Total grid points for FULL run: 4643
 Starting GEE batch extraction...
 GEE extraction complete.
 Starting OpenWeather rain data extraction...
 500/4643 points processed in 2.31 seconds.
 1000/4643 points processed in 6.83 seconds.
 1500/4643 points processed in 11.53 seconds.
 2000/4643 points processed in 15.29 seconds.
 2500/4643 points processed in 19.50 seconds.
 3000/4643 points processed in 23.86 seconds.
 3500/4643 points processed in 27.77 seconds.
 4000/4643 points processed in 32.13 seconds.
 4500/4643 points processed in 35.85 seconds.
 Parallel rain data extraction complete.

 Full Analysis Complete
 Grid points analyzed: 4643
 Saved full results at: /content/drive/MyDrive/final_project/landslide_prediction/nepal_risk_full_run_3000_calls_per_min_6km.csv

Risk Distribution:
risk_level
High        0.708163
Low         0.237993
Moderate    0.053844
Name: proportion, dtype: float64
